In [119]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from plots import visualization
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from helpers_ex5 import sample_data, load_data 
from implementations import standardize, de_standardize
# load data.
height, weight, gender = load_data()

# build sampled x and y.
seed = 1
y = np.expand_dims(gender, axis=1)
X = np.c_[height.reshape(-1), weight.reshape(-1)]
y, X = sample_data(y, X, seed, size_samples=500)
x, mean_x, std_x = standardize(X)
y.shape, x.shape

In [ ]:
y_sub = y
x_sub = build_poly(x, 1)
y_sub.shape, x_sub.shape

In [ ]:
# indices_of_ones = y_sub==1
# y_sub[y_sub==0] = 1
# y_sub[indices_of_ones] = 0
y_sub[:10]

## load and clean boson data 

In [116]:
col_labels.shape

(5000,)

In [120]:
from proj1_helpers import *
from helpers import equalize_predictions

# load the boson data set
data_path = "../dataset/train.csv"
ndata = 5000
y_loaded, x_loaded, col_labels = load_csv_data(data_path, ndata)#sub_sample=False)
y_loaded = y_loaded.reshape((-1, 1))
y_loaded,x_loaded = equalize_predictions(y_loaded,x_loaded)
y_loaded[y_loaded==-1] = 0
y_loaded.shape, x_loaded.shape

((3288, 1), (3288, 30))

In [121]:
x_ = drop_corr_columns(x_loaded)
x_ = fill_with_nan_list(x_, nan_values=[0, -999])
x_, mean_x, std_x = standardize(x_)
x_, y_ = drop_nan_rows(x_, y_loaded)
# x_ = sustitute_nans(x_, substitutions=np.nanmean(x_, axis=0)) 
x_.shape, y_.shape

((783, 21), (783, 1))

In [102]:
# chose the degree
degree = 4
tx_ = build_poly(x_, degree)
tx_.shape

(782, 121)

In [60]:
# # take a subset
# n = 5000
# x_sub = tx_[:n, :]
# y_sub = y_boson[:n]
# y_sub[y_sub==-1] = 0

# y_sub.shape, x_sub.shape

((5000, 1), (5000, 57))

## Logistic Regression

Compute your cost by negative log likelihood.

In [103]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return 1.0 / (1 + np.exp(-t))

In [104]:
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    pred = sigmoid(tx @ w)
    loss = y.T @ (np.log(pred)) + (1 - y).T @ (np.log(1 - pred))
    return np.squeeze(- loss) 

In [105]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    pred = sigmoid(tx @ w)
    
    grad = tx.T @ (pred - y)
    return grad

### Using Gradient Descent
Implement your function to calculate the gradient for logistic regression.

In [108]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    w -= gamma * grad
    return loss, w

Demo!

In [113]:
def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 1000
    threshold = 1e-8
    gamma = 1e-6
    losses = []

    # build tx
#     tx = np.c_[np.ones((y.shape[0], 1)), x]
    tx = x
    w = np.zeros((x.shape[1], 1))

    lowest_loss = float('Inf')
    best_w = -1
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        
        gamma /= 1.2
        
        if loss < lowest_loss:
            lowest_loss = loss
            best_w = w
            
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
#         if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
#             break
    # visualization
    # visualization(y, x[:, 1:], mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent")
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    
    return best_w

w = logistic_regression_gradient_descent_demo(y_sub, x_sub)

Current iteration=0, loss=3465.7359027997313


/home/niccolo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()
/home/niccolo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


Current iteration=100, loss=3223.6638366078955
Current iteration=200, loss=3223.6638332890925
Current iteration=300, loss=3223.6638332890925
Current iteration=400, loss=3223.6638332890925
Current iteration=500, loss=3223.6638332890925
Current iteration=600, loss=3223.6638332890925
Current iteration=700, loss=3223.6638332890925
Current iteration=800, loss=3223.6638332890925
Current iteration=900, loss=3223.6638332890925
loss=3223.6638332890925


In [114]:
y_sub_ = y_sub.copy()
y_sub_[y_sub_== 0] = -1
compute_loss(y_sub_, x_sub, w, costfunc=CostFunction.SUCCESS_RATIO)

0.69279999999999997

### Using penalized logistic regression
Fill in the function below.

In [ ]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss and gradient."""
    num_samples = y.shape[0]
    loss = calculate_loss(y, tx, w) + lambda_ * np.squeeze(w.T.dot(w))
    gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w
    return loss, gradient

In [ ]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
    w -= gamma * gradient
    return loss, w

In [ ]:
def logistic_regression_penalized_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 500
    gamma = 1e-13
    lambda_ = 0.1
    threshold = 1e-6
    losses = []

    # build tx
#     tx = np.c_[np.ones((y.shape[0], 1)), x]
    tx = x
    w = np.zeros((tx.shape[1], 1))

    lowest_loss = float('Inf')
    best_w = -1
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # update lambda
        gamma /= 1.2
        
        if loss < lowest_loss:
            lowest_loss = loss
            best_w = w
            
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent")
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    
    return best_w
    
w = logistic_regression_penalized_gradient_descent_demo(y_sub, x_sub)

In [ ]:
y_sub_ = y_sub.copy()
y_sub_[y_sub_==0] = -1
compute_loss(y_sub_, x_sub, w, costfunc=CostFunction.SUCCESS_RATIO)

In [ ]:
lam = 1e-8

In [ ]:
lam = lam/(1+lam)